In [ ]:
import os,sys,torch,time

# 将本地包路径添加到 sys.path 的最前面
local_packages_path = os.path.abspath("/data/workspace/projects/HupuKiller/local_libs/site-packages")
sys.path.insert(0, local_packages_path)

import torch
from torch.ao.quantization import get_default_qconfig
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx
from torch.ao.quantization import  QConfig,QConfigMapping
from transformers import pipeline,AutoTokenizer
from transformers import BertForSequenceClassification, BertTokenizer
from torch.utils.data import DataLoader, TensorDataset
from torch.ao.quantization.observer import HistogramObserver,default_observer,default_weight_observer


# 项目根目录
project_root_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
# 项目python源码目录
py_src_root_dir = os.path.dirname(os.getcwd())
sys.path.append(py_src_root_dir)

from common.myFile import * 
from customDistilBert import CustomDistilBert 

from distilBertInfer import getDataLoaderAndLabels


# 自定义 HistogramObserver (目前的计算图IR中存在将torch.Size作为HistogramObserver输入的情况，需要特殊处理)
class CustomHistogramObserver(HistogramObserver):
    def forward(self, x_orig):
        # 如果输入是 torch.Size，直接返回,不进行直方图信息统计
        if isinstance(x_orig, torch.Size):
            return x_orig
        else: 
            return super().forward(x_orig)


# custom_qconfig = QConfig(
#     activation=CustomHistogramObserver.with_args(dtype=torch.qint8,reduce_range=True),
#     weight = default_weight_observer
# )

custom_qconfig = get_default_qconfig("x86")


def do_fx_mode_ptq():

    with torch.autograd.profiler.profile(use_cuda=False) as prof:
        device = torch.device('cpu')
        model_path = '/data/workspace/projects/HupuKiller/outputs/DistilBert/fineTune'
        bert_base = BertForSequenceClassification.from_pretrained(model_path).to(device)
        tokenizer = BertTokenizer.from_pretrained(model_path)
        customDistilBert = CustomDistilBert(bert_base).to(device)
        checkpointFilePath = '/data/workspace/projects/HupuKiller/outputs/DistilBert/distillation/model_epoch_4.pth'
        distilCheckpoints = torch.load(checkpointFilePath,map_location=device)
        customDistilBert.load_state_dict(distilCheckpoints)

        model = customDistilBert
        model.eval()

        dataLoader,rawLabels = getDataLoaderAndLabels(batchSize = 8,tokenizer=tokenizer,device=device)
        # qconfig = get_default_qconfig("x86")
        qconfig_mapping = QConfigMapping().set_global(custom_qconfig)

        # 打印 CUDA 操作日志
        # print(prof.key_averages().table(sort_by="cuda_time_total"))

        def calibrate(model: torch.nn.Module, data_loader: DataLoader):
            model.eval()
            with torch.no_grad():
                for batch in dataLoader:
                    batch_input_ids, batch_attention_mask = batch

                    # 新添加逻辑,为了可以通过 torch.ao.quantization.HistogramObserver 的观测
                    batch_attention_mask = batch_attention_mask.float()  # 为了解决某个报错

                    model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)

        example_inputs = (next(iter(dataLoader))[0]) # get an example input
        prepared_model = prepare_fx(model, qconfig_mapping,example_inputs)  # fuse modules and insert observers
        calibrate(prepared_model, dataLoader)  # run calibration on sample data
        quantized_model = convert_fx(prepared_model)  # convert the calibrated model to a quantized model

        return quantized_model

/data/workspace/anaconda/envs/HupuKiller/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ptq_model = do_fx_mode_ptq()

/tmp/ipykernel_197749/93579053.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  distilCheckpoints = torch.load(checkpointFilePath,map_location=device)
/data/workspace/an

In [3]:
type(ptq_model.encoder)

torch.nn.modules.module.Module

In [4]:
encoder_1 = next(ptq_model.encoder.children())

In [5]:
print(type(encoder_1))
print(encoder_1)

<class 'torch.nn.modules.module.Module'>
Module(
  (attention): Module(
    (self): Module(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
    )
    (output): Module(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    )
  )
  (intermediate): Module(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
  )
  (output): Module(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  )
)


In [8]:
print(type(encoder_1.attention.self.value))
print(encoder_1.attention.self.value.weight.dtype)

<class 'torch.nn.modules.linear.Linear'>
torch.float32


In [7]:
for name, child in ptq_model.encoder.named_children():
    print(f"name:{name} ; child: {child}")

name:0 ; child: Module(
  (attention): Module(
    (self): Module(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
    )
    (output): Module(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    )
  )
  (intermediate): Module(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
  )
  (output): Module(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  )
)
name:1 ; child: Module(
  (attention): Module(
    (self): Module(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True

In [26]:
for name, module in ptq_model.named_modules():
    print(f"Module: {name}")
    print(f"  Quantized Weight: {module.weight()}")
    print(f"  Quantized Bias: {module.bias()}")

Module: 


AttributeError: 'GraphModule' object has no attribute 'weight'